# **connect** **to** **drive**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Filter test

In [ ]:
import cv2
import matplotlib.pyplot as plt

image = cv2.imread(image_road)

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

g_image = cv2.GaussianBlur(image_rgb, (31, 31), 0)
b_image = cv2.bilateralFilter(image_rgb, d=20, sigmaColor=80, sigmaSpace=400)

fig, axs = plt.subplots(1, 3, figsize=(15, 10))

axs[0].imshow(image_rgb)
axs[0].set_title('REAL')
axs[1].imshow(g_image)
axs[1].set_title('guass')
axs[2].imshow(b_image)
axs[2].set_title('bilateral')
#plt.tight_layout()
plt.show()


# Dense CRF

In [ ]:
pip install git+https://github.com/lucasb-eyer/pydensecrf.git

In [ ]:
pip install torch torchvision


In [ ]:
import os
import cv2

image_path1 = '/content/drive/My Drive/Colab Notebooks/dense/gan_generated.png'
image_path2 = '/content/drive/My Drive/Colab Notebooks/Clean/Abyek05.png'
image_path3 = '/content/drive/My Drive/Colab Notebooks/Real/Abyek05.jpg'



image_process = cv2.imread(image_path1)
image_mask= cv2.imread(image_path2)
image_grb = cv2.imread(image_path3)
image_real = cv2.cvtColor(image_road, cv2.COLOR_BGR2RGB)


if image_real.shape[:2] != image_process.shape[:2]:
    image_process = cv2.resize(image_process, (image_real.shape[1], image_real.shape[0]), interpolation=cv2.INTER_NEAREST)


# Check if the file exists
if os.path.exists(image_path1) and os.path.exists(image_path2) and os.path.exists(image_path3) :
    print("File exists")
else:
    print("File not found")


In [ ]:
import cv2
import matplotlib.pyplot as plt
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral, create_pairwise_gaussian
import numpy as np
from skimage.color import gray2rgb
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from osgeo import gdal
import pydensecrf.densecrf as dcrf

%matplotlib inline

image_path = "/content/drive/My Drive/Colab Notebooks/OpenCV/normal.png"
image=

def crf(original_image, annotated_image,output_image, use_2d = True):

    if(len(annotated_image.shape)<3):
        annotated_image = gray2rgb(annotated_image).astype(np.uint32)

    cv2.imwrite("testing2.png",annotated_image)
    annotated_image = annotated_image.astype(np.uint32)
    annotated_label = annotated_image[:,:,0].astype(np.uint32) + (annotated_image[:,:,1]<<8).astype(np.uint32) + (annotated_image[:,:,2]<<16).astype(np.uint32)

    colors, labels = np.unique(annotated_label, return_inverse=True)

    colorize = np.empty((len(colors), 3), np.uint8)
    colorize[:,0] = (colors & 0x0000FF)
    colorize[:,1] = (colors & 0x00FF00) >> 8
    colorize[:,2] = (colors & 0xFF0000) >> 16

    n_labels = len(set(labels.flat))

    print("No of labels in the Image are ")
    print(n_labels)


    if use_2d :
        d = dcrf.DenseCRF2D(original_image.shape[1], original_image.shape[0], n_labels)

        U = unary_from_labels(labels, n_labels, gt_prob=0.90, zero_unsure=False)
        d.setUnaryEnergy(U)


        d.addPairwiseGaussian(sxy=(40, 15), compat=46, kernel=dcrf.DIAG_KERNEL, #15,15,46
                          normalization=dcrf.NORMALIZE_SYMMETRIC)

        d.addPairwiseBilateral(sxy=(50, 5), srgb=(1, 1, 1), rgbim=original_image, #40,20
                           compat=42, #46
                           kernel=dcrf.DIAG_KERNEL,
                           normalization=dcrf.NORMALIZE_SYMMETRIC)

    Q = d.inference(2) #3

    MAP = np.argmax(Q, axis=0)


    MAP = colorize[MAP,:]
    cv2.imwrite(output_image,MAP.reshape(original_image.shape))
    return MAP.reshape(original_image.shape)

crfimage = crf(image_real,image_process,"out.png")

plt.subplots(1, 4, figsize=(20, 10))
plt.subplot(1,4,1)
plt.imshow(image_real)
plt.subplot(1,4,2)
plt.imshow(crfimage)
plt.subplot(1,4,3)
plt.imshow(image_process)
plt.subplot(1,4,4)
plt.imshow(image_mask)

# **Inverting Images to NumPy Array**

In [ ]:
import os
import numpy as np
from PIL import Image

def load_images_from_folder(folder, image_size=(700, 700)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if img_path.endswith(".png") or img_path.endswith(".jpg") or img_path.endswith(".jpeg"):
            img = Image.open(img_path).convert('L')
            img = img.resize(image_size)
            img_array = np.array(img)
            images.append(img_array)
    return np.array(images)

input_folder = '/content/drive/My Drive/Colab Notebooks/Noisy'
target_folder = '/content/drive/My Drive/Colab Notebooks/Clean'

input_images = load_images_from_folder(input_folder)
target_images = load_images_from_folder(target_folder)

np.save('/content/drive/My Drive/Colab Notebooks/Noisy/input_images.npy', input_images)
np.save('/content/drive/My Drive/Colab Notebooks/Clean/target_images.npy', target_images)

print(f"Saved {len(input_images)} input images and {len(target_images)} target images as NumPy arrays.")

# **GAN** (**Generative** **Adversarial** **Networks**)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

class NoisyIdealDataset(Dataset):
    def __init__(self, noisy_dir, ideal_dir, noisy_ext='png', ideal_ext='png', transform=None):
        self.noisy_dir = noisy_dir
        self.ideal_dir = ideal_dir
        self.noisy_ext = noisy_ext
        self.ideal_ext = ideal_ext
        self.transform = transform

        self.noisy_images = sorted([f for f in os.listdir(noisy_dir) if f.endswith(noisy_ext)])
        self.ideal_images = sorted([f for f in os.listdir(ideal_dir) if f.endswith(ideal_ext)])

        assert len(self.noisy_images) == len(self.ideal_images), "Mismatch in number of files"

    def __len__(self):
        return len(self.noisy_images)

    def __getitem__(self, idx):
        noisy_filename = self.noisy_images[idx]
        ideal_filename = self.ideal_images[idx]

        noisy_path = os.path.join(self.noisy_dir, noisy_filename)
        ideal_path = os.path.join(self.ideal_dir, ideal_filename)

        noisy_img = Image.open(noisy_path).convert("L")
        ideal_img = Image.open(ideal_path).convert("L")

        resize = transforms.Resize((256, 256))
        noisy_img = resize(noisy_img)
        ideal_img = resize(ideal_img)

        if self.transform:
            noisy_img = self.transform(noisy_img)
            ideal_img = self.transform(ideal_img)

        return noisy_img, ideal_img

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(256*256, 1024),
            nn.ReLU(),

            nn.Linear(1024, 512),
            nn.ReLU(),

            nn.Linear(512, 256),
            nn.ReLU(),

            nn.Linear(256, 256*256),
            nn.Tanh()
        )

    def forward(self, input):
        input = input.view(input.size(0), -1)
        return self.main(input).view(input.size(0), 1, 256, 256)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(256*256, 1024),
            nn.LeakyReLU(0.2),

            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),

            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),

            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        input = input.view(input.size(0), -1)
        return self.main(input)

mean = [0.5]
std = [0.5]

def denormalize(tensor, mean, std):
    denorm_tensor = tensor * std[0] + mean[0]
    return denorm_tensor

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

noisy_dir = '/content/drive/My Drive/Colab Notebooks/Noisy'
ideal_dir = '/content/drive/My Drive/Colab Notebooks/Clean'
dataset = NoisyIdealDataset(noisy_dir, ideal_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator().to(device)
discriminator = Discriminator().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0005, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0005, betas=(0.5, 0.999))

epochs = 400
for epoch in range(epochs):
    for i, (input_img, target_img) in enumerate(dataloader):
        input_img = input_img.to(device)
        target_img = target_img.to(device)

        optimizer_D.zero_grad()
        real_output = discriminator(target_img)
        real_loss = criterion(real_output, torch.ones_like(real_output))
        real_loss.backward()

        fake_img = generator(input_img)
        fake_output = discriminator(fake_img.detach())
        fake_loss = criterion(fake_output, torch.zeros_like(fake_output))
        fake_loss.backward()
        optimizer_D.step()

        d_loss = real_loss + fake_loss

        optimizer_G.zero_grad()
        fake_output = discriminator(fake_img)
        g_loss = criterion(fake_output, torch.ones_like(fake_output))
        g_loss.backward()
        optimizer_G.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs} [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

    if epoch % 20 == 0:
        inverted_img = 1 - denormalize(fake_img, mean, std)
        save_image(inverted_img.data[:7], f'/content/drive/My Drive/Colab Notebooks/result/gan_generated_image_{epoch}.png', nrow=4, normalize=False)

        fig = plt.figure(figsize=(20, 12))
        grid = make_grid(inverted_img, nrow=3)
        plt.imshow(grid.permute(1, 2, 0).cpu().numpy(), cmap='gray')
        plt.axis('off')
        plt.title(f'Generated Images at Epoch {epoch}')
        plt.show()
